## Section 1: Import Libraries and load cifar10 dataset.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
#import additional libraries if needed

In [2]:
def load_cifar10_data(batch_size):
# load both training and test datasets, and transform them to tensors.
  transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  
  return trainloader, testloader

In [3]:
batch_size = 4
trainloader, testloader = load_cifar10_data(batch_size)
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Check the dimensions of a batch:
for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

100%|██████████| 170498071/170498071 [00:03<00:00, 46611553.16it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Image batch dimensions: torch.Size([4, 3, 224, 224])
Image label dimensions: torch.Size([4])


## Section 2: Load Pre-trained Model 

In [4]:
from torchvision import models

model0 = models.alexnet(pretrained=True)
model1 = models.vgg16(pretrained=True)
model2 = models.resnet50(pretrained=True)


# Freeze all pre-trained layers
for param in model0.parameters():
  param.requires_grad = False

# Replace the output layer with your own output layer
model0.classifier[4] = nn.Linear(4096,1024)
model0.classifier[6] = nn.Linear(1024,10)

# Unfreeze the changed layers
for param in model0.classifier[4].parameters(): #changed layers
  param.requires_grad = True
for param in model0.classifier[6].parameters(): 
  param.requires_grad = True


#for vgg

for param in model1.parameters():
  param.requires_grad = False
model1.classifier[3] = nn.Linear(4096,1024)
model1.classifier[6] = nn.Linear(1024,10)
for param in model1.classifier[3].parameters(): 
  param.requires_grad = True
for param in model1.classifier[6].parameters(): 
  param.requires_grad = True


#for ResNet-50



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 184MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed

In [5]:
import torch.nn as nn

for param in model2.parameters():
  param.requires_grad = False

# model2.fc = nn.Linear(2048, 10)
# new_fc = nn.Linear(1000, 10)
# # Connect the layers
# model2 = nn.Sequential(
#     model2.fc,
#     new_fc
# )

model2.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 10))

## Section 3: Define optimizer and loss criterion

In [6]:
#TODO: Declare your hyperparameters here
random_seed = 20
learning_rate = 0.001
num_epochs = 20

# optimizer
optimizer0 = optim.SGD(model0.parameters(), lr=learning_rate, momentum=0.9)
optimizer1 = optim.SGD(model1.parameters(), lr=learning_rate, momentum=0.9)
optimizer2 = optim.SGD(model2.parameters(), lr=learning_rate, momentum=0.9)
# loss
criterion = nn.CrossEntropyLoss()

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)
# Set my pretrained model to be run on GPU
model0 = model0.to(DEVICE)
model1 = model1.to(DEVICE)
model2 = model2.to(DEVICE)
criterion.to(DEVICE)

Device: cuda:0


CrossEntropyLoss()

## Section 4: Train the model

In [8]:
def train_model(model,num_epochs, trainloader, optimizer, criterion):
    for epoch in range(num_epochs):
    
        model= model.train()
        for batch_idx, (features, targets) in enumerate(trainloader):
            features = features.to(DEVICE)
            targets = targets.to(DEVICE)
            
            # Forward and backward propagation
            optimizer.zero_grad()
            output = model(features)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            # Print loss and accuracy every 10 batches
            if batch_idx % 1000 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, batch_idx, len(trainloader), loss.item()))


## Section 5: Test the model

Write a function to test the model using testloader.

In [10]:
def test_model(testloader, model):
    # Set the model to evaluation mode
    model.eval()

    # Initialize counters for correct predictions and total examples
    correct = 0
    total = 0

    # Iterate over the dataloader
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data

            # Move the data to the device
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            # Update counters
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Compute accuracy
    accuracy = 100 * correct / total

    #print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f} %')
    print('Accuracy of the network on the test images for model:  %.2f %%' % (accuracy))

In [ ]:
train_model(model0,num_epochs, trainloader, optimizer0, criterion)
test_model(testloader, model0)

train_model(model1,num_epochs, trainloader, optimizer1, criterion)
test_model(testloader, model1)

train_model(model2,num_epochs, trainloader, optimizer2, criterion)
test_model(testloader, model2)

Epoch [1/20], Step [0/12500], Loss: 2.2539
Epoch [1/20], Step [1000/12500], Loss: 1.8329
Epoch [1/20], Step [2000/12500], Loss: 1.5316
Epoch [1/20], Step [3000/12500], Loss: 0.8016
Epoch [1/20], Step [4000/12500], Loss: 0.5727
Epoch [1/20], Step [5000/12500], Loss: 0.6139
Epoch [1/20], Step [6000/12500], Loss: 1.1881
Epoch [1/20], Step [7000/12500], Loss: 2.0873
Epoch [1/20], Step [8000/12500], Loss: 1.0095
Epoch [1/20], Step [9000/12500], Loss: 1.0340
Epoch [1/20], Step [10000/12500], Loss: 0.1783
Epoch [1/20], Step [11000/12500], Loss: 0.6847
Epoch [1/20], Step [12000/12500], Loss: 1.3464
Epoch [2/20], Step [0/12500], Loss: 0.9374
Epoch [2/20], Step [1000/12500], Loss: 0.5692
Epoch [2/20], Step [2000/12500], Loss: 0.8197
Epoch [2/20], Step [3000/12500], Loss: 2.8599
Epoch [2/20], Step [4000/12500], Loss: 1.8667
Epoch [2/20], Step [5000/12500], Loss: 0.6171
Epoch [2/20], Step [6000/12500], Loss: 0.7177
Epoch [2/20], Step [7000/12500], Loss: 1.8517
Epoch [2/20], Step [8000/12500], Loss